In [ ]:
import csv
from pandas import read_csv
from matplotlib import pyplot as plt
import numpy as np
import os
import sys
import seaborn as sns
import pandas as pd
import math
from numpy import split
import random
import re

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import numpy
from tqdm import tqdm_notebook
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from numpy import array
#from itertools import product
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras import optimizers
from tensorflow import keras
import tensorflow as tf
from numpy.random import seed
from scipy import stats
from IPython.display import clear_output 
import statistics
import keras.backend as K

plt.rcParams['savefig.dpi'] = 300


def maximum(a, b):
      
    if a >= b:
        return a
    else:
        return b

def minimum(a, b):
      
    if a <= b:
        return a
    else:
        return b

#for parsing the units, this function removes the comma and replaces it with dash to avoid splitting the units (as different hyperparameters) when building the hyperparameters list
def removeCommaInUnits(HP):
  units= HP[HP.index('['):HP.index(']')+1]
  if ',' in units:
    units = units.replace(',','-')
  HP=HP.replace(HP[HP.index('['):HP.index(']')+1],units)
  return HP

#this function puts back the comma in its place as it is needed while building our model
def putCommaInUnitsList(HP):
  return [i.replace('-',',') for i in HP] 



def exponential_smoothing(series, alpha):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result
  
def plot_exponential_smoothing(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing - "+attack)
    plt.grid(True);

def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result

def plot_double_exponential_smoothing(series, alphas, betas):
     
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        for beta in betas:
            plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(series.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing - "+attack)
    plt.grid(True)
    plt.show()
    



#stationarity test
def tsplot(y, lags=None, figsize=(12, 7), syle='bmh'):
    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style='bmh'):
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title(attack_title+' - Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
        plt.show()
        
 
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=1):
    # flatten data
    data = train #modified

    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)
 

class BayesianLSTM(LSTM): #inherits LSTM but set training=True which keeps dropout on during test time.
  def call(self, inputs):
    return super().call(inputs, training=True)

# train the model using Bayesian LSTM
def build_model(train, test, n_input, layer, unit, epoch, lr,rdo):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    test_x, test_y=to_supervised(test,n_input)

    # define parameters
    verbose, epochs, batch_size = 0,epoch, 8

    model = Sequential()

    if layer==1:
        model.add(BayesianLSTM(unit[0], input_shape=(n_input, 1),recurrent_dropout=rdo, activation = 'relu'))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True, recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
    elif layer==2:
        model.add(BayesianLSTM(unit[0], activation = 'relu', input_shape=(n_input, 1),return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu' ,recurrent_dropout=rdo))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[1], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
    elif layer==3:
        model.add(BayesianLSTM(unit[0], activation = 'relu', input_shape=(n_input, 1),return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu' ,return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[2], activation = 'relu' ,recurrent_dropout=rdo))
        model.add(RepeatVector(1))
        model.add(BayesianLSTM(unit[2], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[1], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(BayesianLSTM(unit[0], activation = 'relu', return_sequences=True,recurrent_dropout=rdo))
        model.add(TimeDistributed(Dense(100, activation = 'relu')))
        model.add(TimeDistributed(Dense(1)))
          

    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='mse', metrics=['mape'])
    
    #if the loss does not improve after 30 epochs, stop anyway (early stopping)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    # fit network
    train_history= model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_x, test_y),verbose=verbose,callbacks=[es])
    
    loss = train_history.history['loss']
    val_loss = train_history.history['val_loss']

    return model


#forecasts the future trend of a single attack 3 years in advance
def forecast(data, e_data, smoothed, model, min_list,seed, attack_label):
    
    #get best model's hyperparameters
    n_input=min_list[2]
    layer=min_list[3]
    unit=min_list[4]
    lr=min_list[5]
    epoch=min_list[6]
    rdo=min_list[7]

    #build the model using the best hyperparameters using the whole data
    model = build_model(e_data, e_data, n_input, layer, unit, epoch, lr,rdo)

    #Bayesian LSTM performs multiple iterations to create a prediction distribution (i.e., mean and variance)
    n_iterations=10
    predictions_2d=list()
    for r in range(0,n_iterations):

      predictions = list()
      predictions.append(e_data[-1,0]) #for plotting purpose
      #look back in test data
      history=e_data[-n_input:,]

      #predicts 36 months using sliding window
      for i in range(36):
        f_hat=model.predict(history.reshape(1,history.shape[0],history.shape[1]))
        history=numpy.append(history, f_hat[0][0])
        history=history.reshape(history.shape[0],1)
        history=history[1:,]#move the sliding window 1 step
        predictions.append(f_hat[0][0][0])
      predictions_2d.append(predictions)

    predictions_2d = array(predictions_2d)
    predictions_mean=numpy.mean(predictions_2d,axis=0)#mean
    predictions_var=numpy.var(predictions_2d,axis=0)#variance
    std_predictions=numpy.std(predictions_2d,axis=0)#standard deviation
    z=1.96
    confidence_95=z*std_predictions/math.sqrt(len(predictions_mean))# 95% confidence interval

    #calculate the height of data on the plot (plot range) to disable variance/95% conf.interval plot if too large or too small.
    upper=maximum(numpy.amax(predictions_mean),numpy.amax(data))
    lower=minimum(numpy.amin(predictions_mean),numpy.amin(data))
    p_range=upper-lower
    var_plot=True
    max_var=0
    for v in predictions_var:
      if v>p_range:
        var_plot=False
      if v>max_var:
        max_var=v
    if max_var*30<p_range:
      var_plot=False

    conf_plot=True
    max_conf=0
    for conf in confidence_95:
      if conf>p_range:
        conf_plot=False
      if conf>max_conf:
        max_conf=conf
    if max_conf*30<p_range:
      conf_plot=False 


    print(e_data)
    print(predictions_mean)
    
    #Plot the forecast
    x=['2012', '2013','2014', '2015', '2016', '2017', '2018','2019', '2020', '2021','2022','2023','2024','2025']
  #	pyplot.plot(L,test[-n_input:, 0],'b-',label='Actual')
    pyplot.plot(range(len(data)),data,'--', color="blue",label='Data')
    pyplot.plot(range(len(data), len(data)+len(predictions_mean)),predictions_mean,'--', color="red",label='Prediction')
    if conf_plot:
      pyplot.fill_between(range(len(data), len(data)+len(predictions_mean)),(predictions_mean.reshape(-1)- (confidence_95.reshape(-1))), (predictions_mean.reshape(-1)+confidence_95.reshape(-1)), color='green', alpha=0.2,label='95% Confidence')
    if var_plot:
      pyplot.fill_between(range(len(data), len(data)+len(predictions_mean)),(predictions_mean.reshape(-1)- (predictions_var.reshape(-1))), (predictions_mean.reshape(-1)+predictions_var.reshape(-1)), color='green', alpha=0.1,label='Variance')
    plt.xticks([6,18,30,42,54,66,78,90,102,114,126,138,150,162], x) # positions of years on x axis

    plt.ylabel("No of incidents",fontsize=15)
    plt.yticks(fontsize=13)
    plt.legend(loc="best",prop={'size': 11})
    plt.axis('tight')
    plt.grid(True)
    pyplot.xticks(rotation=90,fontsize=13)
    pyplot.title(attack_label+' (U)', y=1.03,fontsize=18)
     
    #save and show the forecast
    images_dir = 'output_forecast/'
    plt.savefig(f"{images_dir}/"+attack_label.replace('/','_')+str(seed)+".png", bbox_inches = "tight")#the name of the file includes the seed
    pyplot.show() 


    #write the data and forecast
    data=data.ravel().tolist()
    predictions_mean=predictions_mean.tolist()

    files_dir = 'output_forecast/'
    with open(files_dir+attack_title.replace('/','_')+'_data_'+str(seed)+'.txt', 'w') as f:#the name of the file includes the seed
      f.write('Data: '+str(data))
      f.write('\nPredictions: '+str(predictions_mean))
      data.extend(predictions_mean)
      f.write('\nData and Predictions: '+str(data))
      f.write('\n95_Confidence: '+str(confidence_95.tolist()))
      f.write('\nVariance: '+str(predictions_var.tolist()))
      f.close()








#read input data
data = read_csv('input_data/T-HMGDN-F-0711-0322.csv', header=0)
attacks=data.columns.values.tolist()
raw_values = data.values
avg_data=0

#name of attack columns (in the dataset) to be forecasted. "All" refers to the number of attacks in all countries combined 
attacks=['DDoS-ALL','Ransomware-ALL', 'Account Hijacking-ALL', 'Zero-day-ALL', 'Malware-ALL',
         'Brute Force Attack-ALL', 'Botnet-ALL','Mentions-MITM', 'Mentions-Dropper', 'Mentions-Adversarial Attack', 'Mentions-Deepfake',
          'Mentions-Supply Chain','Mentions-IoT Device Attack']
done=[] #can be manually appended with already forecasted attacks to avoid repeating the process


#iterate over all attacks to forecast their trend
for attack in attacks:

  #clear_output()
  if attack in done:
    continue
  if not 'ALL' in attack and not 'Mentions' in attack:
    continue;
  if 'Mentions-' in attack:
    attack_title=attack[attack.index('-')+1:]
  else:
    attack_title=attack[0:attack.rindex('-')]


  if len(attack_title)>18:
    attack_title=attack_title.rsplit(' ', 1)[0]



  best_data=None
  best_e_data=None


  # get the optimal model hyperparameters from file
  files_dir = 'output_hyperparameters/'
  with open(f"{files_dir}/"+attack_title.replace('/','_')+'.txt') as f:
      lines = f.readlines()
      f.close()
  HP_str= lines[1]
  HP_str=HP_str[HP_str.index('[')+1:-1]
  HP_str=removeCommaInUnits(HP_str)
  HP=HP_str.split(', ')
  HP=putCommaInUnitsList(HP)
  print('HP:',HP)


  min_mape=999999999
  min_list=[]
  model=None
  c=1


  #read smoothing constants
  alpha=float(HP[0])
  beta=float(HP[1])
  
  #Double Exponential Smoothing
  if not alpha==1 and not beta==1:
    smoothed=double_exponential_smoothing(data[attack], alpha,beta)
    smoothed=smoothed[0:-1] #thanks for the one additional forecast but no thanks,
    plt.plot(smoothed, label="Alpha {}, beta {}".format(alpha, beta))
    plt.title("Double Exponential Smoothing - "+attack_title)
    avg_data=sum(smoothed)/len(smoothed)
  #Exponential Smoothing  
  elif not alpha==1 and beta==1: 
    smoothed=exponential_smoothing(data[attack], alpha)
    plt.plot(smoothed, label="Alpha {}".format(alpha))
    plt.title("Exponential Smoothing - "+attack_title)
    avg_data=sum(smoothed)/len(smoothed)
  #no smoothing  
  elif alpha==1 and beta==1:
    smoothed=data[attack]
    avg_data=smoothed.mean()

  #plot the data
  plt.plot(data[attack], label = "Actual")
  plt.legend(loc="best")
  plt.axis('tight')
  plt.grid(True)
  plt.xlabel("Month")
  plt.ylabel("No of incidents")
  plt.show()

  #stationarity test
  tsplot(smoothed, lags=30)


  smoothed_all=numpy.array([smoothed]).reshape(-1,1)
  prepared_data=smoothed
  scaled_data=numpy.array(prepared_data).reshape(-1,1)

  #cut first part of the data if it is zeros (look for emerging index of the attack)
  e_index=0 #emerging index
  for i in range(len(scaled_data)-2):
    if scaled_data[i,0]>0 and scaled_data[i+1,0]>0 and scaled_data[i+2,0]>0:
        e_index=i
        break
  print('Emerging Index=',e_index)
  scaled_e_data=scaled_data[e_index:]
  print('Length of data after cut=',len(scaled_e_data))
    
  #Get model hyperparameters:
  lag=int(HP[2])#lookback period
  lr=float(HP[5])#learning rate
  epoch=int(HP[6])# epochs
  unit=list(map(int, HP[4].replace('[','').replace(']','').split(', '))) #units
  layer=len(unit) #layers
  rdo=float(HP[7]) #recurrent dropout
  diff_label='not differenced' #no differencing


  #build the set of hyperparameters based on the optimised model
  hp_set=[alpha, beta,lag,layer, unit, lr, epoch,rdo,diff_label]
  print('Hyperparameters:',hp_set)

  #forecast the attack's future trend using 3 seeds 1-3
  for a in range(1,4):
    print('seed:',a)
    seed(a)
    tf.random.set_seed(a)   
    forecast(scaled_data,scaled_e_data,smoothed_all, model,hp_set,a, attack_title)
